In [4]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

# Load datasets
housing_data = fetch_california_housing()
housing_df = pd.DataFrame(housing_data.data, columns=housing_data.feature_names)
housing_df['target'] = housing_data.target

print("Dataset shapes:")
print(f"California Housing: {housing_df.shape}")

# Create train/test splits (80/20)
housing_train, housing_test = train_test_split(housing_df, test_size=0.2, random_state=42)

print("Training set sizes:")
print(f"Housing: {len(housing_train)} samples")

# Convert to TabularDataset
housing_train_data = TabularDataset(housing_train)
housing_test_data = TabularDataset(housing_test)

# Create predictor with Mitra for regression
print("Training Mitra regressor on California Housing dataset...")
mitra_reg_predictor = TabularPredictor(
    label='target',
    path='./mitra_regressor_model',
    problem_type='regression'
)
mitra_reg_predictor.fit(
    housing_train_data.sample(1000), # sample 1000 rows
    hyperparameters={
        'MITRA': {'fine_tune': False}
    },
)

# Evaluate regression performance
mitra_reg_predictor.leaderboard(housing_test_data)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #79-Ubuntu SMP Fri Apr 25 14:51:39 UTC 2025
CPU Count:          128
Memory Avail:       487.19 GB / 503.53 GB (96.8%)
Disk Space Avail:   315.94 GB / 1758.86 GB (18.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in co

Beginning AutoGluon training ...
AutoGluon will save models to "/home/dplaskow/pfn-gift-eval/tabpfn-time-series/gift_eval/plotting/mitra_regressor_model"
Train Data Rows:    1000
Train Data Columns: 8
Label Column:       target
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    498880.45 MB
	Train Data (Original)  Memory Usage: 0.06 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):

Dataset shapes:
California Housing: (20640, 9)
Training set sizes:
Housing: 16512 samples
Training Mitra regressor on California Housing dataset...


	-0.5628	 = Validation score   (-root_mean_squared_error)
	0.8s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'Mitra': 1.0}
	-0.5628	 = Validation score   (-root_mean_squared_error)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1.59s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 5571.1 rows/s (200 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/home/dplaskow/pfn-gift-eval/tabpfn-time-series/gift_eval/plotting/mitra_regressor_model")


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,Mitra,-0.545477,-0.562759,root_mean_squared_error,0.333246,0.035654,0.804209,0.333246,0.035654,0.804209,1,True,1
1,WeightedEnsemble_L2,-0.545477,-0.562759,root_mean_squared_error,0.335339,0.035900,0.805833,0.002093,0.000246,0.001624,2,True,2


In [5]:
import torch, torch.nn as nn

mm = mitra_reg_predictor._trainer.load_model('Mitra')        # or 'TabPFNv2'
pt = mm.model.trainers[0].model               # the underlying torch.nn.Module

In [6]:
pt

Tab2D(
  (x_quantile): Tab2DQuantileEmbeddingX()
  (x_embedding): Tab2DEmbeddingX(
    (x_embedding): Linear(in_features=1, out_features=512, bias=True)
  )
  (y_embedding): Tab2DEmbeddingYRegression(
    (y_embedding): Linear(in_features=1, out_features=512, bias=True)
    (y_mask): Embedding(1, 512)
  )
  (layers): ModuleList(
    (0-11): 12 x Layer(
      (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attention1): MultiheadAttention(
        (q): Linear(in_features=512, out_features=512, bias=True)
        (k): Linear(in_features=512, out_features=512, bias=True)
        (v): Linear(in_features=512, out_features=512, bias=True)
        (o): Linear(in_features=512, out_features=512, bias=True)
      )
      (layer_norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (linear1): Linear(in_features=512, out_features=2048, bias=True)
      (linear2): Linear(in_features=2048, out_features=512, bias=True)
      (layer_norm3): LayerNorm((512,), ep